In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms


batch_size_train = 10000
batch_size_test = 20000
log_interval = 10

num_samples = 512 # number of training samples



# num_samples = 128 # number of training samples
num_samples_test = 100 # number of test samples

new_dim1 = 32 * 1 # first dimension
new_dim2 = 32 * 1 # second dimension


old_dim = 32 # MNIST original dimension


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# transform = transforms.Compose(
    # [transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size_test,
                                         shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



Files already downloaded and verified
Files already downloaded and verified


In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

print(example_data.shape)
print(example_targets.shape)

In [ ]:
# Create training data and test data for binary classification task

from tqdm import tqdm 
training_data = enumerate(train_loader)
test_data = enumerate(test_loader)
(batch_id1, (data_tr_old, target_tr_old)) = next(training_data)
(batch_id2, (data_test_old, target_test_old)) = next(test_data)
print("loaded")


# Classes
# class1 = [2]
# class2 = [9]
class1 = [1]
class2 = [5]

# Pad the image on the right and on the bottom
def filter_digits(data_old, target_old, num, threshold):
    data = torch.zeros([num, 3, new_dim1, new_dim2])
    target = torch.zeros([num])

    num_samples_per_class = int(num / 2)
    sample1 = 0
    sample2 = 0
    attempt = 0
    sample = 0

    while (sample1 < num_samples_per_class) or (sample2 < num_samples_per_class):
        if attempt == threshold:
            print("FAILED: need more samples in batch")
            return (data, target)
        # Balance classes
        is_sample_for_class1 = (target_old[attempt] in class1) and (sample1 < num_samples_per_class)
        is_sample_for_class2 = (target_old[attempt] in class2) and (sample2 < num_samples_per_class)
        if is_sample_for_class1 or is_sample_for_class2: 
            avg = np.zeros(3)
            for c in range(3):
                avg[c] = torch.mean(data_old[attempt][c])
            target[sample] = target_old[attempt]
            for c in range(3):
                for i in range(old_dim):
                    for j in range(old_dim):
                        data[sample][c][i][j] = data_old[attempt][c][i][j] - avg[c]
            if is_sample_for_class1:
                sample1 += 1
            if is_sample_for_class2:
                sample2 += 1
          # Augment sample counts
            sample += 1
        attempt += 1

    target.apply_(lambda x: 1 if (x in class1) else -1)

    data = data.float()
    target = target.float()
    return (data, target)


# Training data and test data
(data_tr_v1, target_tr_v1) = filter_digits(data_tr_old, target_tr_old, num_samples, batch_size_train)
print("Created training data")
(data_test_v1, target_test_v1) = filter_digits(data_test_old, target_test_old, num_samples_test, batch_size_test)
print("Created test data")

torch.save(data_tr_v1, "training-test-data/training_data.txt")


torch.save(target_tr_v1, "training-test-data/training_targets.txt")


torch.save(data_test_v1, "training-test-data/test_data.txt")


torch.save(target_test_v1, "training-test-data/test_targets.txt")





In [ ]:
# Check that classes are balanced
counter = 0
for i in range(num_samples):
    if target_tr[i] == 1:
        counter += 1
print(counter/num_samples)

counter = 0
for i in range(num_samples_test):
    if target_test[i] == 1:
        counter += 1
print(counter/num_samples_test)

print(data_tr.shape)

# for i in range(10):
#     plt.imshow(data_tr[i].permute((1,2,0)))
#     plt.show()
#     print(target_tr[i])
